# Docling PDF Reader example

In [ ]:
import rich
from rich.pretty import pprint
import warnings
import os
from dotenv import load_dotenv

load_dotenv()
warnings.filterwarnings(action="ignore", category=FutureWarning, module="easyocr")
warnings.filterwarnings(action="ignore", category=UserWarning, module="pydantic")

## Basic usage

In order to load PDF data with Docling, we use a `DoclingPDFReader`.

To demonstrate a basic RAG pipeline, we set:
- `export_type` to Markdown, for using the Markdown text format
- `chunk_docs` to False, to just get back the exported documents (one can also use `chunk_docs=True` — in that case, chunking would also be performed internally, so the returned nodes would already be chunked)

In [ ]:
from llama_index.readers.docling.base import DoclingPDFReader

reader = DoclingPDFReader(
    export_type=DoclingPDFReader.ExportType.MARKDOWN,  # Markdown export or native format (JSON)
    chunk_docs=False,  # whether to chunk the docs already within the reader or return the raw content
)
docs = reader.load_data(
    file_path="https://arxiv.org/pdf/2408.09869",  # PDF local path or URL (or iterable thereof)
)

Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

Let's preview a doc sample:

In [ ]:
import rich

md_snippet = docs[0].text[:700]
rich.print(f"{md_snippet}...")

<!-- image -->

## Docling Technical Report

## Version 1.0

Christoph Auer Maksym Lysak Ahmed Nassar Michele Dolfi Nikolaos Livathinos Panos Vagenas Cesar Berrospi Ramis 
Matteo Omenetti Fabian Lindlbauer Kasper Dinkla Lokesh Mishra Yusik Kim Shubham Gupta Rafael Teixeira de Lima 
Valery Weber Lucas Morin Ingmar Meijer Viktor Kuropiatnyk Peter W. J. Staar

AI4K Group, IBM Research Ruschlikon, Switzerland

## Abstract

This technical report introduces Docling , an easy to use, self-contained, MITlicensed open-source package for PDF 
document conversion. It is powered by state-of-the-art specialized AI models for layout analysis (DocLayNet) and 
table structure recognition (TableFormer), and run...

## RAG demo

Next, we get the chunks and preview a chunk sample:

In [ ]:
from llama_index.core.node_parser import MarkdownNodeParser

node_parser = MarkdownNodeParser()
nodes = node_parser.get_nodes_from_documents(documents=docs)
pprint(nodes[7], max_length=2, max_string=250, max_depth=2)

TextNode(
│   id_='6bb282b4-afd7-4c97-9a33-1fed30f046bf',
│   embedding=None,
│   metadata={
│   │   'dl_doc_hash': '556ad9e23b6d2245e36b3208758cf0c8a709382bb4c859eacfe8e73b14e635aa',
│   │   'Header_2': '3.1 PDF backends'
│   },
│   excluded_embed_metadata_keys=['dl_doc_hash'],
│   excluded_llm_metadata_keys=['dl_doc_hash'],
│   relationships={
│   │   <NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(...),
│   │   <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(...),
│   │   ... +1
│   },
│   text='3.1 PDF backends\n\nTwo basic requirements to process PDF documents in our pipeline are a) to retrieve all text content and their geometric coordinates on each page and b) to render the visual representation of each page as it would appear in a PDF vie'+1029,
│   mimetype='text/plain',
│   start_char_idx=5009,
│   end_char_idx=6288,
│   text_template='{metadata_str}\n\n{content}',
│   metadata_template='{key}: {value}',
│   metadata_seperator='\n'
)

Setting up the embed model:

In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

Setting up the vector store:

In [ ]:
from tempfile import TemporaryDirectory
from llama_index.vector_stores.milvus import MilvusVectorStore

vector_store = MilvusVectorStore(
    uri=os.environ.get(
        "MILVUS_URL", default=f"{(tmp_dir := TemporaryDirectory()).name}/milvus_demo.db"
    ),
    collection_name="docling_collection",
    dim=len(embed_model.get_text_embedding("hi")),
    overwrite=True,
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Setting up the index:

In [ ]:
from llama_index.core import StorageContext, VectorStoreIndex

storage_context = StorageContext.from_defaults(vector_store=vector_store)

index = VectorStoreIndex(
    nodes=nodes,
    embed_model=embed_model,
    storage_context=storage_context,
    show_progress=True,
)

Generating embeddings:   0%|          | 0/33 [00:00<?, ?it/s]

Setting up the LLM:

In [ ]:
from llama_index.llms.huggingface_api import HuggingFaceInferenceAPI

HF_TOKEN = os.environ.get("HF_TOKEN")

llm = HuggingFaceInferenceAPI(
    token=HF_TOKEN,
    model_name="mistralai/Mixtral-8x7B-Instruct-v0.1",
)

And now we are ready to ask questions on our document content.

In [ ]:
query_engine = index.as_query_engine(llm=llm)
query_res = query_engine.query("Can I use OCR with Docling?")
pprint(query_res, max_length=5, max_string=70, max_depth=4)

Response(
│   response='\n\nYes, Docling provides optional support for OCR. It relies on EasyOCR'+439,
│   source_nodes=[
│   │   NodeWithScore(
│   │   │   node=TextNode(
│   │   │   │   id_='c5cccef6-6c34-493e-8ebf-7c7276c47e16',
│   │   │   │   embedding=None,
│   │   │   │   metadata={...},
│   │   │   │   excluded_embed_metadata_keys=[...],
│   │   │   │   excluded_llm_metadata_keys=[...],
│   │   │   │   relationships={...},
│   │   │   │   text='OCR\n\nDocling provides optional support for OCR, for example to cover s'+591,
│   │   │   │   mimetype='text/plain',
│   │   │   │   start_char_idx=8983,
│   │   │   │   end_char_idx=9644,
│   │   │   │   text_template='{metadata_str}\n\n{content}',
│   │   │   │   metadata_template='{key}: {value}',
│   │   │   │   metadata_seperator='\n'
│   │   │   ),
│   │   │   score=0.8368496894836426
│   │   ),
│   │   NodeWithScore(
│   │   │   node=TextNode(
│   │   │   │   id_='c10eb76d-8954-4ac4-bc97-f8bd97fdc734',
│   │   │   │   embedding=None,
│   │   │   │   metadata={...},
│   │   │   │   excluded_embed_metadata_keys=[...],
│   │   │   │   excluded_llm_metadata_keys=[...],
│   │   │   │   relationships={...},
│   │   │   │   text='1 Introduction\n\nConverting PDF documents back into a machine-processab'+1800,
│   │   │   │   mimetype='text/plain',
│   │   │   │   start_char_idx=858,
│   │   │   │   end_char_idx=2728,
│   │   │   │   text_template='{metadata_str}\n\n{content}',
│   │   │   │   metadata_template='{key}: {value}',
│   │   │   │   metadata_seperator='\n'
│   │   │   ),
│   │   │   score=0.7604262232780457
│   │   )
│   ],
│   metadata={
│   │   'c5cccef6-6c34-493e-8ebf-7c7276c47e16': {
│   │   │   'dl_doc_hash': '556ad9e23b6d2245e36b3208758cf0c8a709382bb4c859eacfe8e73b14e635aa',
│   │   │   'Header_2': 'OCR'
│   │   },
│   │   'c10eb76d-8954-4ac4-bc97-f8bd97fdc734': {
│   │   │   'dl_doc_hash': '556ad9e23b6d2245e36b3208758cf0c8a709382bb4c859eacfe8e73b14e635aa',
│   │   │   'Header_2': '1 Introduction'
│   │   }
│   }
)